In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentencepiece
!pip install torch
!pip install transformers


     |████████████████████████████████| 1.2MB 8.4MB/s 
     |████████████████████████████████| 1.8MB 7.3MB/s 
     |████████████████████████████████| 2.9MB 35.8MB/s 
     |████████████████████████████████| 890kB 50.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=dd7100e12d77d370931292e5bfe6493d43f0668a0f227d32afb7ee191b09cb97
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
#import pretrained models 
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
#model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-tw")

In [4]:
#import further libraries that will be needed
import nltk
from nltk.translate.bleu_score import SmoothingFunction
import pandas as pd

In [5]:
def translate(input_sentence, model=model, **kwargs):
    """
    Translate the sentence to the target language using the provided model and any keyword arguments provided.
    input_sentence: List of sentences
    model: model to use for the translation.
    
    Returns:
    List of corresponding translations for the target language
    """

    input_id = tokenizer.encode(input_sentence, return_tensors="pt")
    beam_output = model.generate(input_id,  **kwargs)
    return tokenizer.decode(beam_output[0], skip_special_tokens=True)

In [6]:
def bleuscore_dataframe(dataframe, weights=(0.85,0.15,0,0), smoothFn=None,auto_reweigh=False):
    """
    compute the bleu score of an entire dataframe dataframe should have a predictions and 
    references column
    dataframe: pd.DataFrame - should have "predictions" and "references" columns
    weights is a 4-tuple summing up to 1. indicates weights for unigrams, bigrams etc
    smoothFn:int {0,7}. refer to nltk API for details
    """
    preds = []
    for line in dataframe['predictions'].values.tolist(): 
        line_clean = line.strip().split()
        preds.append(line_clean)
    refs = []
    for line in dataframe["references"].values.tolist(): 
        line_clean = str(line).strip().split()
        refs.append(line_clean)
    refs_2 = [list([(item)]) for item in refs]  # putting it in a form for the nltk bleu method
    if smoothFn in range(8):
        smoothFunc = SmoothingFunction()
        smoothFunc = eval(f"smoothFunc.method{smoothFn}")
    else:
        smoothFunc = smoothFn
    BLEUscore = nltk.translate.bleu_score.corpus_bleu(refs_2, preds, weights=weights,
                                                      smoothing_function=smoothFunc,auto_reweigh=auto_reweigh)
    return BLEUscore

In [29]:
#read the parrarel dataset to be used for checking scores
pairs = pd.read_csv("/content/drive/My Drive/GhanaNLP/DATA/GhanaNLP_data_001.csv")

In [31]:
#remove rows with empty references
print("totals records in dataset BEFORE removing empty rows {}".format(pairs.shape[0]))
pairs.dropna(inplace=True)
print("totals records in dataset AFTER removing empty rows {}".format(pairs.shape[0]))

totals records in dataset BEFORE removing empty rows 695
totals records in dataset AFTER removing empty rows 695


In [8]:
def predict_sample_sentences(parallel_sentences, sample_size=100):
  """
    Takes a dataframe of parrarel eng-twi sentences and translate the english sentences to target language


    parallel_sentences: pd.DataFrame - should have "English" and "references" columns with "English" sentences in column index 0 and "references in col index 1
    sample_size : number of sentences in the dataframe to translate, default is 100 sentences
    
    Return:
    Returns a dataframe with a third column "Predictions" added which contain the translated sentences
    """

  test_pairs = parallel_sentences.sample(sample_size)
  test_pairs.columns= ["English", "references"]
  test_pairs["predictions"] = test_pairs["English"].apply(translate)
  return test_pairs

In [32]:
#translate some sample sentences
sample_test_pairs = predict_sample_sentences(pairs,sample_size=pairs.shape[0])

In [42]:
#check that all references have predictions
sample_test_pairs.describe().T

,count,unique,top,freq
English,695,664,How are you?,6
references,695,668,Me dɔ wo,4
predictions,695,647,Wo ho te dɛn?,8


In [44]:
#take a preview of 6 of the sentences
sample_test_pairs.head(6)

,English,references,predictions
39,God’s Grace,Onyame adom,Onyankopɔn De Ne Ho Gye Mu
667,Please tell me where the bus stop is.,Mepa wo kyɛw kyerɛ me baabi a wɔfa kai no,Mesrɛ wo ka baabi a bɔs gyinabea no wɔ kyerɛ me.
48,Come home.,Bra fie.,Bra fie.
456,He carved a Buddhist image out of wood.,ɔde dua no nwenee Budhafoɔ mfoni.,Ɔde dua yɛɛ Buddhafo honi.
420,I thought Kwaku needed help.,"Na mefiri sɛ, Kwaku hia mmoa.",Ná misusuw sɛ Kwaku hia mmoa.
0,What is going on here?,Ɛdeɛn na ɛrekɔso wɔ aha?,Dɛn na ɛrekɔ so wɔ ha?


In [45]:
#check some sample bluescores with different paramaters
#1. default bleu score weights on 4-grams with weights=(0.25,0.25,0.25,0.25) and no smoothing function and no auto_reweigh - Option to re-normalize the weights uniformly.
bleuscore_dataframe(sample_test_pairs[["references","predictions"]])

0.24937600624285972

In [46]:
#2. put all emphasis on uni-grams (1.0)  and smoothing function of 7 and with auto_reweigh
bleuscore_dataframe(sample_test_pairs[["references","predictions"]], weights=(1,0,0,0),smoothFn=7, auto_reweigh=True)

0.5628157501706027

In [47]:
#3. default bleu score weights on 4-grams with weights=(0.25,0.25,0.25,0.25)  and smoothing function of 7 and with auto_reweigh
bleuscore_dataframe(sample_test_pairs[["references","predictions"]],smoothFn=7, auto_reweigh=True)

0.49797381435055743

In [49]:
#4. put some emphasis on the uni-gram with weights=(0.58,0,0,0)  and smoothing function of 7 and with auto_reweigh = True
bleuscore_dataframe(sample_test_pairs[["references","predictions"]], weights=(0.58,0,0,0),smoothFn=7, auto_reweigh=True)

0.7164937735242538